In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from bs4 import BeautifulSoup
import pandas as pd
import time

# 1. 셀레니움 드라이버 실행
driver = webdriver.Chrome()
driver.get("https://taas.koroad.or.kr/sta/acs/exs/typical.do?menuId=WEB_KMP_OVT_UAS_ASA")
time.sleep(3)

# 2. 조회 버튼 클릭
driver.find_element(By.CSS_SELECTOR, "#contents > div.tbl_search_option > div > a").click()
time.sleep(2)

# 3. iframe 전환 + 테이블 로딩 대기
WebDriverWait(driver, 10).until(EC.frame_to_be_available_and_switch_to_it((By.NAME, "eosFrame")))
WebDriverWait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#OctagonGrid tr")))

# 4. soup 파싱
soup = BeautifulSoup(driver.page_source, "html.parser")

# 5. 테이블 행 추출
table_rows = soup.select("#OctagonGrid tr")
data_rows = soup.select("#dataSession tr")

# 6. 시간대 headers 수동 지정
headers = ['00~02시', '02~04시', '04~06시', '06~08시', '08~10시',
           '10~12시', '12~14시', '14~16시', '16~18시', '18~20시',
           '20~22시', '22~24시']

data = []
curr_topcat = ""
curr_midcat = ""
구분 = ""

# 7. 테이블 내용 파싱
for table_row, data_row in zip(table_rows, data_rows):
    table_cells = table_row.find_all("td")
    data_cells = data_row.find_all("td")

    table_texts = [cell.text.strip() for cell in table_cells]
    data_texts = [cell.text.strip() for cell in data_cells]

    if not table_texts or not data_texts:
        continue

    if len(table_cells) == 3:
        curr_topcat = table_texts[0]
        curr_midcat = table_texts[1]
        구분 = table_texts[2]
    elif len(table_cells) == 2:
        curr_midcat = table_texts[0]
        구분 = table_texts[1]
    elif len(table_cells) == 1:
        구분 = table_texts[0]
    else:
        continue

    if len(data_texts) != len(headers):
        continue

    for 시간대, 값 in zip(headers, data_texts):
        data.append([curr_topcat, curr_midcat, 구분, 시간대, 값])

driver.quit()

# 8. 데이터프레임 생성 및 저장
df = pd.DataFrame(data, columns=["사고유형대분류", "사고유형중분류", "구분", "시간대", "값"])
df.to_csv("시간대별_사고통계.csv", index=False, encoding='utf-8-sig')
print("✅ 시간대별_사고통계.csv 저장 완료!")

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#contents > div.tbl_search_option > div > a"}
  (Session info: chrome=135.0.7049.42); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF73B455335+78597]
	GetHandleVerifier [0x00007FF73B455390+78688]
	(No symbol) [0x00007FF73B2091AA]
	(No symbol) [0x00007FF73B25F149]
	(No symbol) [0x00007FF73B25F3FC]
	(No symbol) [0x00007FF73B2B2467]
	(No symbol) [0x00007FF73B28712F]
	(No symbol) [0x00007FF73B2AF2BB]
	(No symbol) [0x00007FF73B286EC3]
	(No symbol) [0x00007FF73B2503F8]
	(No symbol) [0x00007FF73B251163]
	GetHandleVerifier [0x00007FF73B6FEEED+2870973]
	GetHandleVerifier [0x00007FF73B6F9698+2848360]
	GetHandleVerifier [0x00007FF73B716973+2967875]
	GetHandleVerifier [0x00007FF73B47017A+188746]
	GetHandleVerifier [0x00007FF73B47845F+222255]
	GetHand